In [1]:
%matplotlib inline

In [2]:
import numpy
import pandas as pd
from easydict import EasyDict as edict
import matplotlib.pyplot as plt

In [3]:
import argparse
import datetime
import itertools
import os
import pprint
import random
import sys

import dateutil
import dateutil.tz
import numpy as np
import torch

from shutil import copyfile
from utils.config import cfg, cfg_from_file
from utils.utils import mkdir_p

In [4]:
# Loading arguments
args = edict({
    "cfg_file": './config/colon_death/cox_colon_death.yml',
    "gpu_id": -1,
    "manual_seed": 1234,
})

cfg_from_file(args.cfg_file)
# cfg

In [5]:
# Timestamp and config load

# Create Timestamp
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime("%Y_%m_%d_%H_%M_%S")
cfg.TIMESTAMP = timestamp
# Create Output Dir
output_dir = "results/%s_%s_%s" % (cfg.DATA.DATASET, cfg.CONFIG_NAME, timestamp)
cfg.OUTPUT_DIR = output_dir
mkdir_p(output_dir)
# Save config file
copyfile(args.cfg_file, os.path.join(output_dir, "config.yml"))


'results/colon_death_cox_colon_death_2021_11_04_20_40_46/config.yml'

In [ ]:
# Timestamp and config load
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime("%Y_%m_%d_%H_%M_%S")
print(f"Timestamp: {timestamp}")
cfg.TIMESTAMP = timestamp
output_dir = "results/%s_%s_%s" % (cfg.DATA.DATASET, cfg.CONFIG_NAME, timestamp)
cfg.OUTPUT_DIR = output_dir
output_dir
mkdir_p(output_dir)
copyfile(args.cfg_file, os.path.join(output_dir, "config.yml"))
# print("Using config:")
# pprint.pprint(cfg)

In [5]:
# Setting seeds for reproducibility
print(f"\nPyTorch/Random seed: {args.manual_seed}")
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(args.manual_seed)
np.random.seed(args.manual_seed)
torch.manual_seed(args.manual_seed)
if cfg.CUDA:
    torch.cuda.manual_seed_all(args.manual_seed)


PyTorch/Random seed: 1234


NameError: name 'cfg' is not defined

In [45]:
param_lr = [float(i) for i in cfg.TRAIN.LR]
param_l2_coeff = [float(i) for i in cfg.TRAIN.L2_COEFF]
# if cfg.TRAIN.MODEL == "emd":
#     param_prior = [float(i) for i in cfg.EMD.PRIOR]
#     params = list(itertools.product(param_lr, param_l2_coeff, param_prior))
# else:
#     params = list(itertools.product(param_lr, param_l2_coeff))

# Mike 
p = ([float(i) for i in cfg.TRAIN.LR][1], [float(i) for i in cfg.TRAIN.L2_COEFF][1])

In [46]:
# test_cindices = []
# split_nbrs = eval(cfg.DATA.SPLITS)
# split_nbrs = split_nbrs or [0, 1, 2, 3, 4]

# Mike 
split_nb = 0

In [47]:
cfg.PARAMS = "LR" + str(p[0]) + "_L2" + str(p[1])
cfg.TRAIN.LR = p[0]
cfg.TRAIN.L2_COEFF = p[1]

In [48]:
# from trainer import get_algo
# algo = get_algo(split_nb)
from trainer.likelihoodtrainer import LikelihoodTrainer
algo = LikelihoodTrainer(split_nb) 

{'CONFIG_NAME': 'cox_colon_death',
 'CUDA': False,
 'DATA': {'ADD_CENS': False,
          'DATASET': 'colon_death',
          'DEATH_AT_CENSOR_TIME': False,
          'NO_CENSORED_DATA': False,
          'PATH': 'datasets/',
          'SPLITS': '[0, 1, 2, 3, 4]'},
 'GPU_ID': '0',
 'OUTPUT_DIR': 'results/colon_death_cox_colon_death_2021_10_27_16_21_26',
 'PARAMS': 'LR0.001_L21e-05',
 'TIMESTAMP': '2021_10_27_16_21_26',
 'TRAIN': {'BATCH_NORM': True,
           'BATCH_SIZE': 128,
           'DROP_HIDDEN': '[0.5, 0.5, 0.5]',
           'DROP_INPUT': 0.2,
           'L2_COEFF': 1e-05,
           'LAYER_SIZES': '[100, 100, 100]',
           'LOSS_TYPE': 'cox_loss_basic',
           'LR': 0.001,
           'MAX_EPOCH': 1000,
           'MODEL': 'likelihood',
           'PATIENCE': 20},
 'VERBOSE': False}

Dataset colon_death info:
Nb. col: 48
Nb unique t: 780
Min t: 23
Max t: 3329
Nb of event: Counter({0: 477, 1: 452})


In [50]:
algo.model

MLP(
  (layers): ModuleList(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=48, out_features=100, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Linear(in_features=100, out_features=100, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=100, out_features=100, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (output): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:
train, val, test = self.get_data_random_split()
self.before_train(train)
split_id = f"split{self.split}"

scores = {}

train_err_history = []
val_err_history = []
train_cindex_history = []
val_cindex_history = []

In [49]:
from utils.config import cfg, cfg_from_file
score, concat_pred_test = algo.run()

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [ ]:


from utils.utils import mkdir_p
val_cindex = score['val']['c_index']
test_cindex = score['test']['c_index']
print(f"\nVal c_index: {val_cindex}")
print(f"Test c_index: {test_cindex}")

opt_val_cindices.append(val_cindex)
opt_test_cindices.append(test_cindex)